In [2]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np

/home/aasmund_mjos/Downloads/sintef/mrst-rag-project/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
vector_embedding_model = SentenceTransformer('microsoft/codebert-base')

No sentence-transformers model found with name microsoft/codebert-base. Creating a new one with mean pooling.


In [22]:
code_snippet = """
%bismilah yukkk 20 April 2025
%sisa Pb aja yg agak anehh, sm oil rate ko start 20k yeah not 40k, but okay
%
%mismatch OOIP < 5%
%mismatch OGIP < 5%
%mismatch OWIP ~ 20% !!!! WARNING !!!!
%harusnya gada air

%% Add Modules
mrstModule add ad-blackoil ad-core incomp ad-props mrst-gui solvent deckformat

%% 1-Setup Reservoir Grid & Rock Properties
% Definisikan jumlah sel (X, Y, Z)
nx = 7; ny = 7; nz = 3;
Lx = 3500 * ft; Ly = 3500 * ft; % Panjang & Lebar
% Definisikan ketebalan per layer
Lz = [20, 30, 50] * ft; 
% Buat grid dengan ketebalan beda-beda
G = tensorGrid( ...
    linspace(0, Lx, nx+1), ...
    linspace(0, Ly, ny+1), ...
    [0, cumsum(Lz)]); % Layer Z dengan ketebalan beda
% Hitung geometri grid
G = computeGeometry(G);

% Mendapatkan Indeks Layer
[I, J, K] = gridLogicalIndices(G);

% Set Nilai Homogen Sesuai Layer**
poro = 0.3 * ones(G.cells.num, 1); % Semua porositas 0.3
permX = zeros(G.cells.num, 1);
permY = zeros(G.cells.num, 1);
permZ = zeros(G.cells.num, 1);
% Layer 1
permX(K == 1) = 500 * milli * darcy;
permY(K == 1) = 500 * milli * darcy;
permZ(K == 1) = 50 * milli * darcy;
% Layer 2
permX(K == 2) = 50 * milli * darcy;
permY(K == 2) = 50 * milli * darcy;
permZ(K == 2) = 50 * milli * darcy;
% Layer 3
permX(K == 3) = 200 * milli * darcy;
permY(K == 3) = 200 * milli * darcy;
permZ(K == 3) = 25 * milli * darcy;

% **Gabungkan ke dalam struktur rock**
rock = struct('perm', [permX, permY, permZ], 'poro', poro);


%% 2-Setup Fluid & Solvent Gas Properties
%- Initiate Fluid PVT Data
 
[~, ~, fluid, ~, ~] = setupSPE9();


fluid = addSolventProperties(fluid, 'rhoSS' , 1.8687*kilogram/meter^3, ...      
                                    'mixPar', 1                 , ...           
                                    'muS'   , 0.0172*centi*poise     );

                                                                                       
%model3Ph = GenericBlackOilModel(G, rock, fluid, ...
%                             'disgas', true, ...   % Enable dissolved gas
%                             'vapoil', false, ...  % Disable vaporized oil
%                             'water', true, ...    % Enable water
%                             'oil', true, ...      % Enable oil
%                             'gas', true);         % Enable gas
                         
                         
model4Ph = BlackOilSolventModel(G, rock, fluid, 'dynamicEndPointScaling', false);


%% 3-Setup Wells & Injection Schedule
%* Convert into Field Units
%================= VOLUME =================%
m3_to_ft3 = 35.3147;          % Dikali -> dari m3 ke ft3
m3_to_stb = 6.2898;                % Dikali -> dari m3 ke STB (1 m3 * 6.2898 = STB)
m3_to_scf = 35.3147;              % Dikali -> dari m3 ke Mscf (1 m3 * 35.3147 = Mscf)
rb_to_ft3 = 5.615;                 % Dikali -> dari RB to ft3
ft3_to_rb = 1/rb_to_ft3;           % Dibagi -> dari ft3 ke RB
stb_to_m3 = 1 / m3_to_stb;        % Dibagi -> dari STB ke m3 (STB / 6.2898 = m3)
mscf_to_m3 = 1*1000 / m3_to_scf; % Dibagi -> dari Mscf ke m3 (Mscf / 35.3147 = m3)

%================= PRESSURE =================%
pa_to_psi = 0.000145038;      % Dikali -> dari Pascal ke psi (Pa * 0.000145038 = psi)


%* Initiate Wells Array
W = []; % Inisialisasi W sebagai array kosong
day = 86400;         % seconds in a day
year = 365*day;      % seconds in a year

timeW   = 0.5*year;                                 % Duration of water injection periods
timeWAG = 4*timeW;                                  % Duration of WAG period

% Set Parameters
rateSolvent  = 500 * mscf_to_m3 / 86400;            % Convert 1000 MSCF/D to m³/s for gas (solvent)
rateWater    = 500 * stb_to_m3 / 86400;             % Convert 1000 STB/D to m³/s for water
bhp          = 1000*psia;                           % Producer bottom-hole pressure

%* Add Producer
W = verticalWell(W, G, rock, ...
    7, 7, 1:3, ...                                  % Lokasi producer (X, Y, Z)
    'Type', 'bhp', ...
    'Val', bhp, ...                                 % Set BHP produksi oil 1000 psi
    'comp_i', [0 1 0 0], ...
    'Sign', -1, ...    
    'Name', 'Producer');                            % Producer: flow keluar (-1)


%* Add Water-Injector
W = verticalWell(W, G, rock, ...
    1, 1, 1:3, ...                                  % Lokasi injector (X, Y, Z)
    'Type', 'rate', ...
    'Val', rateWater, ...                           % Set laju injeksi 12,000 bbl/d to m3/s
    'comp_i', [1 0 0 0], ...
    'Sign', 1, ...    
    'Name', 'Water-Injector');                      % Injector: flow masuk (+1)

%* Add CO2-Injector
W = verticalWell(W, G, rock, ...
    1, 1, 1:3, ...                                  % Lokasi injector (X, Y, Z)
    'Type', 'rate', ...
    'Val', rateSolvent, ...            % Set laju injeksi 12,000 MSCF/d to m³/s
    'comp_i', [0 0 0 1], ...                        % Komposisi gas
    'Sign', 1, ...                                  % Injector: flow masuk (+1)
    'Name', 'Solvent-Injector');                    % Nama Injector


% Schedule WAG dengan dua injeksi
dt1 = 60*day; % Timestep size for first water injection period
dt2 = 30*day; % Timestep size for WAG cycles and second water injection period
useRampUp = true; % Use ramp up
nCycles = 4; % Four WAG cycles
scheduleWAG = makeWAGschedule(W, nCycles, 'time', timeWAG, ...
                              'dt', dt2, 'useRampup', useRampUp);

tvec1 = rampupTimesteps(timeW, dt1);
tvec2 = rampupTimesteps(timeW, dt2, 0);
scheduleWAG.step.val = [tvec1; scheduleWAG.step.val; tvec2];
scheduleWAG.step.control = [2*ones(numel(tvec1),1); ...
                            scheduleWAG.step.control; ...
                            2*ones(numel(tvec2),1)];

% Menampilkan schedule injeksi
disp(scheduleWAG);

                     
%% 4-Define Model & Initial State, and Simulate Model
%*Initiating Reservoir Pressure for Each Layer (psia)
p_init = zeros(G.cells.num, 1);
p_init(K == 1) = 3584.399 * psia;
p_init(K == 2) = 3594.149 * psia;
p_init(K == 3) = 3609.776 * psia;

s0 = repmat([0.2, 0.8, 0, 0], [G.cells.num, 1]);

% Define Model
model4Ph.extraStateOutput = true;


% Check Bubble Point Pressure (Psia)
rs0 = repmat(247.0488, [G.cells.num, 1]); %dalam RB/BBL
%---%
Bubble_Point = fluid.pb(rs0);
fprintf('Bubble Point Pressure: %.2f Psia\n', Bubble_Point * pa_to_psi);


%state0 = struct('pressure', p_init, 's', s0, 'rs', rs0, 'rv', 0);
state0 = initResSol(G, p_init, [0.2, 0.8, 0, 0]);
state0.wellSol = initWellSolAD(W, model4Ph, state0);


% Plot Result
%fn = getPlotAfterStep(state0, model4Ph, scheduleWAG, ...
%                     'plotWell', true, 'plotReservoir', true);


% Use compiled, iterative linear solver
mrstModule add linearsolvers
lsol = selectLinearSolverAD(model4Ph);
                 
[wellSolsWAG, statesWAG, reports] ...
    = simulateScheduleAD(state0, model4Ph, scheduleWAG, 'LinearSolver', lsol); %'afterStepFn', fn);



%% 5-Calculate OOIP

%- Calculate Original Oil In Place (OOIP), Original Gas In Place (OGIP), and Original Water In Place (OWIP)
[ooip, ogip, owip] = computeOriginalInPlace(G, rock, fluid, state0);

%- Calculate Pore Volume (units check!)
pv_m3 = poreVolume(G, rock);       % Still in m³
pv_ft3 = pv_m3 * m3_to_ft3;        % Convert to ft³
pv_rb = sum(pv_ft3) * ft3_to_rb;   % Convert ft³ ke RB (Total PV)

%- Calculate Total Hidrocarbon Pore Volume
sO = state0.s(:, 2); % So
pv_hc = sum(pv_ft3 .* sO) * ft3_to_rb;


%* Print Result
fprintf('========================\n');
fprintf('Total Hidrocarbon Pore Volume: %.2f RB\n', pv_hc);
fprintf('Total Pore Volume: %.2f RB\n', pv_rb);
fprintf('------------------------\n');
fprintf('Original Oil In Place (OOIP): %.2f STB\n', ooip);
fprintf('Original Gas In Place (OGIP): %.2f SCF\n', ogip);
fprintf('Original Water In Place (OWIP): %.2f STB\n', owip);
fprintf('========================\n');



%% 6-Visualize Result
% **Show 3D Reservoir**
%- Constructing Data for Visualization in plotToolbar
data = struct();
data.Porosity = rock.poro;                   % Porositas
data.Kx = rock.perm(:, 1) / (milli * darcy); % Permeabilitas horizontal (Kx)
data.Ky = rock.perm(:, 2) / (milli * darcy); % Permeabilitas horizontal (Ky)
data.Kz = rock.perm(:, 3) / (milli * darcy); % Permeabilitas vertikal (Kz)
data.Pressure = state0.pressure / psia;      % Tekanan dalam psia
data.SaturationWater = state0.s (:, 1);      % Saturasi air (sw)
data.SaturationOil = state0.s(:, 2);         % Saturasi minyak (sO)
data.SaturationGas = state0.s(:, 3);         % Saturasi gas (sG)

%figure; plotToolbar(G, states);
figure;
 plotToolbar(G, data,'EdgeColor', 'k', 'LineWidth', 0.2); % Menampilkan data reservoir secara interaktif

% Tambahkan sumur ke dalam plot
hold on;
plotWell(G, W, 'color', 'g', 'LineWidth', 2);
hold off;
 
% **Plot Hasil Simulasi**
disp('Simulasi selesai.');
figure;
plotWellSols(wellSolsWAG, cumsum(scheduleWAG.step.val)); % Plot produksi sumur

%plotWellSols(wellSolsWAG, cumsum(WAG_schedule.step.val)); % Plot produksi sumur

% **Interactive visualization of reservoir and well simulation results
%figure('Position', [df(1:2), 600,800])
figure('Position', [100, 100, 600, 600])
plotToolbar(G, statesWAG); %axis equal tight;

% Tambahkan sumur ke dalam plot
hold on;
plotWell(G, W, 'color', 'g', 'LineWidth', 2);
hold off;

%%
%=== Inisialisasi Umum ===%
nSteps = numel(scheduleWAG.step.val);
time_days = zeros(nSteps, 1);
convFactor_STB = 6.2898 * 86400;     % Konversi m^3/s ke STB/day
convFactor_SCF = 35.3147 * 86400;  % Konversi m3/s ke SCF/day

%=== Average Pressure ===%
% Extract reservoir pressure from each state in the simulation
pRes = cellfun(@(x) x.pressure, statesWAG, 'UniformOutput', false);

% Initialize array to store average pressure for each time step
avgpRes = zeros(numel(statesWAG), 1);

% Loop through all time steps to calculate average reservoir pressure
for t = 1:numel(statesWAG)
    avgpRes(t) = mean(pRes{t});  % Compute the average pressure at time step t
end

%=== Laju Produksi Minyak (STB/day) ===%
prod_bhp = zeros(nSteps, 1);
for i = 1:nSteps
    ws = wellSolsWAG{i};
    for j = 1:length(ws)
        if strcmp(ws(j).name, 'Producer')
            prod_bhp(i) = abs(ws(j).bhp) * pa_to_psi;
            break;
        end
    end
    if isfield(statesWAG{i}, 'time')
        time_days(i) = statesWAG{i}.time / 86400;
    end
end

%=== Laju Produksi Minyak (STB/day) ===%
oilRate_STB_day = zeros(nSteps, 1);
for i = 1:nSteps
    ws = wellSolsWAG{i};
    for j = 1:length(ws)
        if strcmp(ws(j).name, 'Producer')
            oilRate_STB_day(i) = abs(ws(j).qOs) * convFactor_STB;
            break;
        end
    end
    if isfield(statesWAG{i}, 'time')
        time_days(i) = statesWAG{i}.time / 86400;
    end
end

%=== Laju Produksi Gas (SCF/day) ===%
gasRate_SCF_day = zeros(nSteps, 1);
for i = 1:nSteps
    ws = wellSolsWAG{i};
    for j = 1:length(ws)
        if strcmp(ws(j).name, 'Producer')
            gasRate_SCF_day(i) = abs(ws(j).qGs) * convFactor_SCF;
            break;
        end
    end
    if isfield(statesWAG{i}, 'time')
        time_days(i) = statesWAG{i}.time / 86400;
    end
end

%=== Laju Produksi Air (STB/day) ===%
waterRate_STB_day = zeros(nSteps, 1);
for i = 1:nSteps
    ws = wellSolsWAG{i};
    for j = 1:length(ws)
        if strcmp(ws(j).name, 'Producer')
            waterRate_STB_day(i) = abs(ws(j).qWs) * convFactor_SCF;
            break;
        end
    end
    if isfield(statesWAG{i}, 'time')
        time_days(i) = statesWAG{i}.time / 86400;
    end
end


%=== Kumulatif Produksi Minyak & Recovery Factor ===%
cumOil_STB = cumtrapz(time_days, oilRate_STB_day);
RF = cumOil_STB / ooip;



%====== Plot Grafik ======%

%=== Plot Laju Produksi Minyak ===%
figure;
plot(time_days, oilRate_STB_day, '-o', 'LineWidth', 1.5);
xlabel('Waktu (hari)');
ylabel('Laju Produksi Minyak (STB/hari)');
title('Laju Produksi Minyak - Producer');
grid on;

%=== Plot Laju Produksi Minyak ===%
figure;
plot(time_days, gasRate_SCF_day, '-o', 'LineWidth', 1.5);
xlabel('Waktu (hari)');
ylabel('Laju Produksi Gas(SCF/hari)');
title('Laju Produksi Gas - Producer');
grid on;

%=== Plot Laju Produksi Air ===%
figure;
plot(time_days, waterRate_STB_day, '-o', 'LineWidth', 1.5);
xlabel('Waktu (hari)');
ylabel('Laju Produksi Air (STB/hari)');
title('Laju Produksi Air - Producer');
grid on;

%=== Plot Average Pressure ===%
figure;
plot(cumsum(scheduleWAG.step.val)/day, avgpRes/6895, 'r', 'LineWidth', 2);
xlabel('Time (days)');
ylabel('Average Reservoir Pressure (psi)');
title('Average Reservoir Pressure vs Time');
grid on;

%=== Plot Producer BHP Pressure ===%
figure;
plot(cumsum(scheduleWAG.step.val)/day, prod_bhp, 'r', 'LineWidth', 2);
xlabel('Time (days)');
ylabel('Producer BHP (psi)');
title('Producer Bottom Hole Pressure vs Time');
grid on;

%=== Plot Cumulative Oil & Recovery Factor ===%
figure;
subplot(2,1,1);
plot(time_days, cumOil_STB, 'LineWidth', 2);
title('Cumulative Oil Production');
xlabel('Time (Days)');
ylabel('Cumulative Oil (STB)');
grid on;

subplot(2,1,2);
plot(time_days, RF, 'LineWidth', 2);
title('Recovery Factor (RF)');
xlabel('Time (Days)');
ylabel('Recovery Factor');
grid on;


%=== Rata-rata Tekanan Reservoir ===%
%ave_p = zeros(nSteps, 1);
%for i = 1:nSteps
%    if ~isempty(statesWAG{i}) && isfield(statesWAG{i}, 'pressure')
%        ave_p(i) = mean(statesWAG{i}.pressure) / psia;
%    else
%        ave_p(i) = NaN;
%    end
%end

%=== Sisa Minyak di Reservoir (STB) ===%
%remaining_oil_STB = zeros(nSteps, 1);
%for i = 1:nSteps
%    if ~isempty(statesWAG{i}) && isfield(statesWAG{i}, 's')
%        sO = statesWAG{i}.s(:,2);
%        remaining_oil_STB(i) = sum(sO .* pv_ft3) / 5.615;  % Konversi ke STB
%    else
%        remaining_oil_STB(i) = NaN;
%    end
%end

%=== Plot Remaining Oil ===%
%figure;
%plot(cumsum(scheduleWAG.step.val)/day, remaining_oil_STB, 'b', 'LineWidth', 2);
%xlabel('Time (days)');
%ylabel('Remaining Oil (STB)');
%title('Remaining Oil in Reservoir');
%grid on;
"""

readme_snippet = """
Models and examples that extend the MRST AD-OO framework found in the ad-core module to black-oil problems. More specifically, the module adds additional models that implement the black-oil equations for multiphase, miscible, compressible flow. Included in the module are also a wide variety of examples validating the solvers against a commercial simulator on standard test cases.
The module includes single-phase, two-phase and three-phase solvers. The three-phase solvers has optional support for problems where gas can dissolve into oil (Rs) and/or oil can vaporize into the gas phase (Rv). The solvers support source terms, boundary conditions and complex wells with changing controls, production limits and multiple segments. Standard benchmark cases (SPE1, SPE9) are included together with comparison results from a commercial simulator.
"""

text_snippet = """
Currently, I'm working on modeling WAG-CO2 using black oil solvent. The fluid data used in this model is based on the SPE9 model. However, some errors occurred while simulating the model. The model required the krO data from the fluid.
How can I define the krO into the fluid property? Hereby, I attach the script file.
"""

In [7]:
df = pd.read_pickle("mrst_repository_embeddings.pkl")
df

,file_path,content,author,email,message,embedding,metadata
0,README.md,![MRST Logo](mrst.png)\n## What is this?\nThis...,Olav Møyner,olav.moyner@gmail.com,Update README to reflect current naming and URLs,"[-0.07085521, -0.10219722, -0.01694617, -0.017...","{'source': 'README.md', 'file_path': 'README.m..."
1,core/Contents.m,% Root directory of MATLAB Reservoir Simulatio...,Olav Møyner,olav.moyner@gmail.com,Update subtree,"[-0.033725966, -0.09336751, -0.009516157, -0.0...","{'source': 'core/Contents.m', 'file_path': 'co..."
2,core/README.txt,The core module offers data structures and rou...,Olav Møyner,olav.moyner@gmail.com,Add 'core/' from commit '9ca69848af3b75a91e9b2...,"[-0.04186531, -0.101500906, -0.010480802, -0.0...","{'source': 'core/README.txt', 'file_path': 'co..."
3,autodiff/ad-blackoil/README.txt,Models and examples that extend the MRST AD-OO...,Olav Møyner,olav.moyner@gmail.com,Add 'autodiff/' from commit '371f2eb780d70a2f7...,"[-0.050007593, -0.05514423, 0.009908324, -0.03...","{'source': 'autodiff/ad-blackoil/README.txt', ..."
4,autodiff/ad-core/README.txt,Object-oriented framework for solvers based on...,Olav Møyner,olav.moyner@gmail.com,Add 'autodiff/' from commit '371f2eb780d70a2f7...,"[-0.02854652, -0.0613468, -0.011980764, -0.028...","{'source': 'autodiff/ad-core/README.txt', 'fil..."
...,...,...,...,...,...,...,...
401,autodiff/compositional/models/natvars/sequenti...,% UTILS\n%\n% Files\n% correctSequentialBHP ...,Olav Møyner,olav.moyner@gmail.com,Update subtree,"[9.016995e-05, -0.0037776113, -0.024296455, -0...",{'source': 'autodiff/compositional/models/natv...
402,co2lab/deprecated/co2lab-old-module/solvers/eq...,% UTILS\n%\n% Files\n% getPhaseFluxAndProps_...,Olav Møyner,olav.moyner@gmail.com,Update subtree,"[-0.022706145, -0.09859328, -0.030754592, 0.03...",{'source': 'co2lab/deprecated/co2lab-old-modul...
403,core/utils/inout/eclipse/deckinput_simple/priv...,% PRIVATE\n% Low-level support routines for ...,Olav Møyner,olav.moyner@gmail.com,Update subtree,"[0.004663852, -0.08249073, -0.037824266, 0.035...",{'source': 'core/utils/inout/eclipse/deckinput...
404,model-io/deckformat/experimental/deprecated/fl...,% Files\n% blackOilPVT.m - Evalua...,Olav Møyner,olav.moyner@gmail.com,Update subtree,"[-0.04230161, -0.11225589, -0.024259279, 0.004...",{'source': 'model-io/deckformat/experimental/d...
